In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [54]:
'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4
'''


pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_16164/COMPINT'
nx=128
ny=192
ncomp=4


'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4
'''

print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_16164/COMPINT


In [55]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 128, 192, 4) , mean = 109.39675903320312 , std = 35.61451071813423 , Max = 220.0 , Min = 25.0 , type=float64
t_C_0 shape: (1, 128, 192) , mean = 84.84549967447917 , std = 13.900578727495837 , Max = 132.0 , Min = 25.0 , type=float64
t_C_1 shape: (1, 128, 192) , mean = 85.0880126953125 , std = 12.606826448452495 , Max = 121.0 , Min = 48.0 , type=float64
t_C_2 shape: (1, 128, 192) , mean = 165.98876953125 , std = 12.757215107069019 , Max = 220.0 , Min = 111.0 , type=float64
t_C_3 shape: (1, 128, 192) , mean = 101.66475423177083 , std = 10.126685555500803 , Max = 161.0 , Min = 60.0 , type=float64


In [56]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (128, 192) , mean = -26.335245768229168 , std = 10.126685555500803 , Max = 33 , Min = -68 , type=int8


In [57]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen24576
size124588
size216004
ratio1= 1.00048828125
ratio2= 0.651204427083


In [58]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 24576
mse = 0.0


In [59]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 128, 192, 4) , mean = 108.69923909505208 , std = 35.966330954264 , Max = 215.0 , Min = 29.0 , type=float64
 channel0 shape: (128, 192) , mean = -44.7247314453125 , std = 11.520664052239832 , Max = 12 , Min = -99 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 16781 Bytes
ratio1= 1.00048828125
ratio2= 0.682820638021
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -44.518473307291664 , std = 10.03542637035589 , Max = 0 , Min = -79 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 16352 Bytes
ratio1= 1.00048828125
ratio2= 0.665364583333
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = 39.0230712890625 , std = 10.607311253353359 , Max = 87 , Min = -9 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 16548 Bytes
ratio1= 1.00048828125
ratio2= 0.67333984375
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = -26.98291015625 , std = 9.239112447776424 , Max = 26 , Min = -71 , type=int8
   olen: 2457

dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -45.219278971354164 , std = 16.89791473901418 , Max = 7 , Min = -98 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18728 Bytes
ratio1= 1.00048828125
ratio2= 0.762044270833
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = 39.98291015625 , std = 17.42394515711876 , Max = 93 , Min = -23 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18928 Bytes
ratio1= 1.00048828125
ratio2= 0.770182291667
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = -27.713094075520832 , std = 14.006697771117798 , Max = 40 , Min = -88 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18028 Bytes
ratio1= 1.00048828125
ratio2= 0.733561197917
dlen= 24576
mse = 0.0



9
  t_C shape: (1, 128, 192, 4) , mean = 107.7522684733073 , std = 38.19487944552595 , Max = 214.0 , Min = 42.0 , type=float64
 channel0 shape: (128, 192) , mean = -46.955891927083336 , std = 13.412507321712331 , Max = 12 , Min

   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18891 Bytes
ratio1= 1.00048828125
ratio2= 0.768676757812
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = -24.799357096354168 , std = 14.719012114287054 , Max = 28 , Min = -72 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17477 Bytes
ratio1= 1.00048828125
ratio2= 0.711140950521
dlen= 24576
mse = 0.0



16
  t_C shape: (1, 128, 192, 4) , mean = 111.67027791341145 , std = 31.967001704535527 , Max = 207.0 , Min = 37.0 , type=float64
 channel0 shape: (128, 192) , mean = -38.105997721354164 , std = 13.967055843060551 , Max = -4 , Min = -91 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17365 Bytes
ratio1= 1.00048828125
ratio2= 0.706583658854
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -37.534383138020836 , std = 13.887602997586367 , Max = -2 , Min = -80 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17403 Bytes
ratio1= 1.00048828125
ratio2= 0.708129882812
dl

dlen= 24576
mse = 0.0



23
  t_C shape: (1, 128, 192, 4) , mean = 109.79627482096355 , std = 36.160344943401405 , Max = 227.0 , Min = 34.0 , type=float64
 channel0 shape: (128, 192) , mean = -42.2177734375 , std = 17.68325101322113 , Max = 2 , Min = -94 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18505 Bytes
ratio1= 1.00048828125
ratio2= 0.752970377604
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = -41.9100341796875 , std = 17.08113404316747 , Max = -3 , Min = -83 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18336 Bytes
ratio1= 1.00048828125
ratio2= 0.74609375
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = 37.083577473958336 , std = 15.819583100300243 , Max = 99 , Min = -11 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18057 Bytes
ratio1= 1.00048828125
ratio2= 0.734741210938
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = -25.770670572916668 , std = 11.088389071753477 , Max = 20 , Min = -

In [60]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.00048828125
   ratio_2= 0.6674906412760416
 
2
   ratio_1= 1.00048828125
   ratio_2= 0.55328369140625
 
3
   ratio_1= 1.00048828125
   ratio_2= 0.6828511555989584
 
4
   ratio_1= 1.00048828125
   ratio_2= 0.6531270345052084
 
5
   ratio_1= 1.00048828125
   ratio_2= 0.7527872721354166
 
6
   ratio_1= 1.00048828125
   ratio_2= 0.7476806640625
 
7
   ratio_1= 1.00048828125
   ratio_2= 0.6856791178385416
 
8
   ratio_1= 1.00048828125
   ratio_2= 0.7636922200520834
 
9
   ratio_1= 1.00048828125
   ratio_2= 0.6751200358072916
 
10
   ratio_1= 1.00048828125
   ratio_2= 0.693023681640625
 
11
   ratio_1= 1.00048828125
   ratio_2= 0.6944173177083334
 
12
   ratio_1= 1.00048828125
   ratio_2= 0.6151224772135416
 
13
   ratio_1= 1.00048828125
   ratio_2= 0.73126220703125
 
14
   ratio_1= 1.00048828125
   ratio_2= 0.7509256998697916
 
15
   ratio_1= 1.00048828125
   ratio_2= 0.7626749674479166
 
16
   ratio_1= 1.00048828125
   ratio_2= 0.6911112467447916
 
17
   ratio_1= 1.00048828